In [8]:
import mysql.connector
import os
import subprocess


In [9]:
db_status = "new"
db = None
db_table_list = []
db_config = {
    "host": "127.0.0.1",
    "port": 3306,
    "user": "root",
    "password": "my-secret-pw",
    "database": "aziendadb"
}

In [14]:
def db_connect():
    global db_status
    global db
    if db_status != "connected":
        db = mysql.connector.connect(
            host=db_config["host"],
            port=db_config["port"],
            user=db_config["user"],
            password=db_config["password"],
            database=db_config["database"]
        )
        db_status = "connected"
        print(db_status)

In [18]:
db_connect()

connected


In [5]:
def db_import( sql_path ):
    db_connect()
    if db_status == "connected" and os.path.exists(sql_path):
        print("ok")
        config = db_config
        command = f"mysql -h {config['host']} -P {config['port']} -u {config['user']} -p{config['password']} {config['database']} < {sql_path}"
        subprocess.run(command, shell=True, check=True)
        

In [7]:
db_import( "../aziendadb.sql" )

ok


mysql: [Warning] Using a password on the command line interface can be insecure.
ERROR 1050 (42S01) at line 30: Table 'access_administration' already exists


CalledProcessError: Command 'mysql -h 127.0.0.1 -P 3306 -u root -pmy-secret-pw aziendadb < ../aziendadb.sql' returned non-zero exit status 1.

In [6]:
def table_list():
    global db_table_list
    table_list = []
    db_connect()
    # Creazione di un cursore
    cursor = db.cursor()
    # Query per ottenere l'elenco delle tabelle
    query = "SHOW TABLES"
    # Esecuzione della query
    cursor.execute(query)
    # Recupero dei risultati
    tables = cursor.fetchall()
    # Stampare l'elenco delle tabelle
    #print("Tabelle nel database aziendadb:")
    for table in tables:
        #print(table[0])
        table_list = table_list + [ table[0] ]
    # Chiusura del cursore e della connessione
    cursor.close()
    db_table_list = table_list
    return table_list
print(table_list())

DatabaseError: 2003 (HY000): Can't connect to MySQL server on '127.0.0.1:3306' (111)

In [ ]:
def print_table_contenent( table_name ):
    table_list()
    if table_name in db_table_list:
        print("table exists")
        db_connect()
        # Creazione di un cursore
        cursor = db.cursor()
        # Nome della tabella da cui recuperare i dati
        #table_name = "CUSTOMER"
        # Query per selezionare tutti i dati dalla tabella
        query = f"SELECT * FROM {table_name}"
        # Esecuzione della query
        cursor.execute(query)
        # Recupero dei risultati
        rows = cursor.fetchall()
        # Stampare il contenuto della tabella
        print(f"Contenuto della tabella {table_name}:")
        for row in rows:
            print(row)
        # Chiusura del cursore e della connessione
        cursor.close()

        

In [ ]:
print_table_contenent( "user" )

table exists
Contenuto della tabella user:
(1, 'Federico', 'Villata', 'Ilvillo', 'rocchino16cm', 'vllfrc98s21', 2147483647, 'federicovillata@gmail.com', 'via rossi 2, Torino', '21/11/1998', 'M', '0', '0', '0')
(2, None, None, 'Luca', 'password', None, None, None, None, None, None, None, None, None)
(3, 'Francesco', 'Bianco', 'Fra', 'password', 'Frnc99x72', 11111111, 'francesco@gmail.com', 'via torino 5, Chivasso', '5/5/1872', 'M', '0', '0', '0')
(4, 'Roberta', 'Troia', 'robi', 'chimica', 'rbttr98dqq', 22109832, 'robertatroia@yahoo.it', 'via ciao 11, torino', '21/11/1998', 'F', '0', '0', '0')
(5, 'Federico', 'bianchi', 'Fede', 'Password', 'asddsdsdasd', 124325365, 'federicovillata1998@gmail.com', 'via rossi 5, chieri', '21/11/1998', 'M', '0', '0', '0')
(6, 'carlo', 'cracco', 'carlo', 'oassword', '1', 1, '1', '1', '1', '1', '1', '1', '1')


In [ ]:
def add_row_to_table(table_name, data):
    try:
        db_connect()  # Assicurati che la connessione al database sia attiva
        if db_status == "connected":
            # Creazione di un cursore
            cursor = db.cursor()
            # Query per inserire una nuova riga nella tabella
            query = f"INSERT INTO {table_name} (column1, column2, column3) VALUES (%s, %s, %s)"  # Sostituisci con i nomi delle colonne e i valori
            # Esecuzione della query
            cursor.execute(query, data)
            # Applica le modifiche al database
            db.commit()
            print(f"Riga aggiunta alla tabella {table_name}")
            # Chiusura del cursore
            cursor.close()
        else:
            print("Connessione al database non riuscita.")
    except mysql.connector.Error as err:
        print(f"Errore di MySQL: {err}")
        db.rollback()  # Annulla le modifiche in caso di errore

    

In [ ]:
new_row_data = ("valore1", "valore2", "valore3")  # Sostituisci con i valori da inserire
# Chiamata alla funzione per aggiungere una riga alla tabella
add_row_to_table("nome_tabella", new_row_data)  # Sostituisci "nome_tabella" con il nome della tua tabella

In [ ]:
def update_row_in_table(table_name, data, condition_columns):
    try:
        db_connect()  # Assicurati che la connessione al database sia attiva
        if db_status == "connected":
            # Creazione di un cursore
            cursor = db.cursor()
            # Creazione della stringa di condizione
            condition_string = " AND ".join([f"{col} = %s" for col in condition_columns])
            # Query per aggiornare una riga nella tabella
            query = f"UPDATE {table_name} SET "
            query += ", ".join([f"{col} = %s" for col in data.keys()])
            query += f" WHERE {condition_string}"
            # Esecuzione della query
            cursor.execute(query, list(data.values()) + condition_columns)
            # Applica le modifiche al database
            db.commit()
            print(f"Riga aggiornata nella tabella {table_name}")
            # Chiusura del cursore
            cursor.close()
        else:
            print("Connessione al database non riuscita.")
    except mysql.connector.Error as err:
        print(f"Errore di MySQL: {err}")
        db.rollback()  # Annulla le modifiche in caso di errore

# Dati per l'aggiornamento della riga
table_name = "nome_tabella"  # Sostituisci con il nome della tua tabella
data = {"colonna1": "nuovo_valore1", "colonna2": "nuovo_valore2"}  # Sostituisci con i nuovi valori
condition_columns = ["colonna_condizione1", "colonna_condizione2"]  # Sostituisci con le colonne di condizione

# Chiamata alla funzione per aggiornare una riga nella tabella
update_row_in_table(table_name, data, condition_columns)

In [ ]:
def delete_row_from_table(table_name, condition_columns):
    try:
        db_connect()  # Assicurati che la connessione al database sia attiva
        if db_status == "connected":
            # Creazione di un cursore
            cursor = db.cursor()
            # Creazione della stringa di condizione
            condition_string = " AND ".join([f"{col} = %s" for col in condition_columns])
            # Query per eliminare una riga dalla tabella
            query = f"DELETE FROM {table_name} WHERE {condition_string}"
            # Esecuzione della query
            cursor.execute(query, condition_columns)
            # Applica le modifiche al database
            db.commit()
            print(f"Riga eliminata dalla tabella {table_name}")
            # Chiusura del cursore
            cursor.close()
        else:
            print("Connessione al database non riuscita.")
    except mysql.connector.Error as err:
        print(f"Errore di MySQL: {err}")
        db.rollback()  # Annulla le modifiche in caso di errore

# Colonne di condizione per l'eliminazione della riga
condition_columns = ["colonna_condizione1", "colonna_condizione2"]  # Sostituisci con le colonne di condizione
table_name = "nome_tabella"  # Sostituisci con il nome della tua tabella

# Chiamata alla funzione per eliminare una riga dalla tabella
delete_row_from_table(table_name, condition_columns)